# Plot paired and unpaired synapses

In [ ]:
# Load up all of the libraries we need

import copy
import pickle
from colour import Color

from deriva.core import HatracStore, ErmrestCatalog, get_credential
from synspy.analyze.pair import SynapticPairStudy, ImageGrossAlignment, transform_points

import synapse_utils

import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go


# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
credential = get_credential("synapse.isrd.isi.edu")
ermrest_catalog = ErmrestCatalog('https', 'synapse.isrd.isi.edu', 1, credential)
hatrac_store = HatracStore('https', 'synapse.isrd.isi.edu', credentials=credential)

Lets start by getting a list of all of the studies currently in the server. We will end up with a list of studies, where each study has been classified by the study type based on the protocol that was followed. 

In [ ]:
# Get the current list of studies from the server.
study_entities = synapse_utils.get_synapse_studies()

print('Identified %d studies' % len(study_entities))

protocol_types = {
    'PrcDsy20160101A': 'aversion',
    'PrcDsy20170613A': 'conditioned',
    'PrcDsy20170615A': 'unconditioned',
    'PrcDsy20170613B': 'fullcycle-control',
    'PrcDsy20171030A': 'groundtruth-control',
    'PrcDsy20171030B': 'interval-groundtruth-control'
}

studies = {}

# Lets make some convienent subsets of studies. These are just lists of study names.
aligned = []
all = []
learners = []
nonlearners = []

for i in study_entities:
    all.append(i['Study'])
    studies[i['Study']] = i
    if protocol_types[i['Protocol']] == 'aversion':
        if i['Learner'] == True:
            i['Type'] = 'learner'
            learners.append(i['Study'])
        else:
            i['Type'] = 'nonlearner'
            nonlearners.append(i['Study'])
    else:
        i['Type'] = protocol_types[i['Protocol']]
        
    try:
        i['Alignment'] = ImageGrossAlignment.from_image_id(ermrest_catalog, i['BeforeImageID'])
    except ValueError:  # Alignments missing....
        continue
    except NotImplementedError:
        print('Alignment Code Failed for study: {0}'.format(i['Study']))
        continue
    else:
        aligned.append(i['Study'])
        

    
print('{0} learners'.format(len(learners)))
print('{0} non-learners'.format(len(nonlearners)))
print('{0} aligned'.format(len(aligned)))

Pick a study that we want to look at and set the parameters for the sweep.

In [ ]:
syn_pair_radii = (4, 300.0,) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

study_list = learners + nonlearners

Now calculate the distances and save the results into dataframes. We are going to orignize things so that we have a copy of studies for each radii that we have, and put them in a top level dictionary.

In [ ]:
# We will have one dictonary of studies per radius
syn_pairs = {r: {} for r in syn_pair_radii}
print('Finding pairs for {0} studies'.format(len(study_list)))

for syn_study_id in study_list:
    print('Processing study {0}'.format(syn_study_id))
    study = SynapticPairStudy.from_study_id(ermrest_catalog, syn_study_id)
    study.retrieve_data(hatrac_store)
        
    s1_to_s2, s2_to_s1 = study.syn_pairing_maps(syn_pair_radii, syn_dx_core_ratio, syn_core_max_ratio)

    # get results for different radii, store them in a dictonary of pandas
    for i,r in enumerate(syn_pair_radii):
        syn_pairs[r][syn_study_id] = copy.deepcopy(studies[syn_study_id])
        r_study = syn_pairs[r][syn_study_id]
        
        unpaired1 = study.get_unpaired(s1_to_s2[i,:], study.s1)
        unpaired2 = study.get_unpaired(s2_to_s1[i,:], study.s2)
        paired1, paired2 = study.get_pairs(s1_to_s2[i,:], study.s1, study.s2)
        
        r_study['Radius'] = r
        r_study['Study'] = syn_study_id
    
        r_study['UnpairedBefore'] =  \
            pd.DataFrame(unpaired1[:,0:5], columns=['z','y','x','core','hollow'])
        r_study['UnpairedBefore'].DataType = 'UnpairedBefore'
        r_study['UnpairedBefore'].Study = syn_study_id
        r_study['UnpairedBefore'].Radius = r
        
        r_study['UnpairedAfter'] = \
            pd.DataFrame(unpaired2[:,0:5], columns=['z','y','x','core','hollow'])
        r_study['UnpairedAfter'].DataType = 'UnpairedAfter'
        r_study['UnpairedAfter'].Study = syn_study_id
        r_study['UnpairedAfter'].Radius = r
        
        r_study['PairedBefore'] = \
            pd.DataFrame(paired1[:,0:5], columns=['z','y','x','core','hollow'])
        r_study['PairedBefore'].DataType = 'PairedBefore'
        r_study['PairedBefore'].Study = syn_study_id
        r_study['PairedBefore'].Radius = r
        
        r_study['PairedAfter'] = \
            pd.DataFrame(paired2[:,0:5], columns=['z','y','x','core','hollow'])
        r_study['PairedAfter'].DataType = 'PairedAfter'
        r_study['PairedAfter'].Study = syn_study_id
        r_study['PairedAfter'].Radius = r

<H1> Align synapses across images

In [ ]:
for aligned_study in aligned:
    for r in syn_pair_radii:
        study = syn_pairs[r][aligned_study]
        image_obj = studies[aligned_study]['Alignment']
        
        alignmentpts = pd.DataFrame([studies[aligned_study][pt] for pt in ['AlignP0', 'AlignP1', 'AlignP2']])
        print(alignmentpts)
        study['AlignmentPts'] = pd.DataFrame(transform_points(image_obj.M, alignmentpts.loc[:,['x','y','z']]),
                                             columns=['x', 'y', 'z'])
        print(study['AlignmentPts'])
        study['AlignedPairedBefore'] = \
            pd.DataFrame(transform_points(image_obj.M, study['PairedBefore'].loc[:,['x','y','z']]),       
                         columns=['x','y','z'])
        study['AlignedPairedBefore']['core'] = study['PairedBefore']['core']
        study['AlignedPairedBefore'].DataType = 'AlignedPairedBefore'
        study['AlignedPairedBefore'].Study = study['PairedBefore'].Study
        study['AlignedPairedBefore'].Radius = r
        
        study['AlignedPairedAfter'] =  \
            pd.DataFrame(transform_points(image_obj.M, study['PairedAfter'].loc[:,['x','y','z']]),
                        columns=['x','y','z'])    
        study['AlignedPairedAfter']['core'] = study['PairedAfter']['core']
        study['AlignedPairedAfter'].DataType = 'AlignedPairedAfter'
        study['AlignedPairedAfter'].Study = study['PairedAfter'].Study
        study['AlignedPairedAfter'].Radius = r
            
        study['AlignedUnpairedBefore'] = \
            pd.DataFrame(transform_points(image_obj.M, study['UnpairedBefore'].loc[:,['x','y','z']]),
                        columns=['x','y','z'])
        study['AlignedUnpairedBefore']['core'] = study['UnpairedBefore']['core']
        study['AlignedUnpairedBefore'].DataType = 'AlignedUnpairedBefore'
        study['AlignedUnpairedBefore'].Study = study['UnpairedBefore'].Study
        study['AlignedUnpairedBefore'].Radius = r
        
        study['AlignedUnpairedAfter'] =  \
            pd.DataFrame(transform_points(image_obj.M, study['UnpairedAfter'].loc[:,['x','y','z']]),
                        columns=['x','y','z'])   
        study['AlignedUnpairedAfter']['core'] = study['UnpairedAfter']['core']
        study['AlignedUnpairedAfter'].DataType = 'AlignedUnpairedAfter' 
        study['AlignedUnpairedAfter'].Study = study['UnpairedAfter'].Study
        study['AlignedUnpairedAfter'].Radius = r

<h1> Dump out studies

In [ ]:
dumpfile = 'pairs-dump.pkl'

with open(dumpfile,'wb') as fo:
    pickle.dump(syn_pairs,fo)
    

<h1> Load previously paired Studies

In [ ]:
dumpfile = 'pairs-dump.pkl'

with open(dumpfile,'rb') as fo:
    syn_pairs = pickle.load(fo)

print('Restored radii: {0}'.format(list(syn_pairs)))

print('Restored {0} studies'.format(len(syn_pairs[list(syn_pairs)[0]])))



In [ ]:
# make this an iterator

def group_studies(grouping):
    if grouping = 'Region'
    #go over each radius
    for r in syn_pairs:
        study[]
        for s in r['Region 1']
            groups[''{0}.{1}''.format(Region1,Region2)] = s
    return groups
    
    

<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7 = [],[],[],[],[],[],[]

for syn_study_id in study_list:
    for r in syn_pair_radii:
        r_study = syn_pairs[r][syn_study_id]
        c1.append(r_study['Study'])
        c2.append(r_study['Type'])
        c3.append(r_study['Radius'])
        c4.append(len(r_study['UnpairedBefore'].index))
        c5.append(len(r_study['UnpairedAfter'].index))
        c6.append(len(r_study['PairedBefore'].index))
        c7.append(len(r_study['PairedAfter'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
# Get the maximum radius
r = max(syn_pairs)

pair_distance = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}

for k, v in syn_pairs[max(syn_pairs)].items():
    before = v['PairedBefore']
    after = v['PairedAfter']
    t = v['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))

hist_data_l = go.Histogram(x=pair_distance['learner'], name='Learners Radias {0}'.format(r))
hist_data_nl = go.Histogram(x=pair_distance['learner'], name='Nonlearners Radias {0}'.format(r))
    
hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=[hist_data_l, hist_data_nl], layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
plot_radius = 4
sl = syn_pairs[plot_radius]

core_ratio = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}
core = { 'learner': pd.Series(), 'nonlearner' : pd.Series()}

for k, v in sl.items():
    before = v['PairedBefore']
    after = v['PairedAfter']
    t = v['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

hist_data_l = go.Histogram(x=core_ratio['learner'], name='Learners Radias {0}'.format(r))
hist_data_nl = go.Histogram(x=core_ratio['learner'], name='Nonlearners Radias {0}'.format(r))
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=[hist_data_l, hist_data_nl], layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H1> Unpaired Centroids

Compute the centroids of the unpaired syapses before/after.  Also find the vector between the before and after centroids.  

In [ ]:
# Select the radius that we want to calculate for
plot_radius = 4
sl = syn_pairs[plot_radius]

# calculate centroids
lbx = [v['UnpairedBefore']['x'].mean() for k,v in sl.items() if v['Type'] == 'learner']
lby = [v['UnpairedBefore']['y'].mean() for k,v in sl.items() if v['Type'] == 'learner']
lbz = [v['UnpairedBefore']['z'].mean() for k,v in sl.items() if v['Type'] == 'learner']

lax = [v['UnpairedAfter']['x'].mean() for k,v in sl.items() if v['Type'] == 'learner']
lay = [v['UnpairedAfter']['y'].mean() for k,v in sl.items() if v['Type'] == 'learner']
laz = [v['UnpairedAfter']['z'].mean() for k,v in sl.items() if v['Type'] == 'learner']

nbx = [v['UnpairedBefore']['x'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']
nby = [v['UnpairedBefore']['y'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']
nbz = [v['UnpairedBefore']['z'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']

nax = [v['UnpairedAfter']['x'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']
nay = [v['UnpairedAfter']['y'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']
naz = [v['UnpairedAfter']['z'].mean() for k,v in sl.items() if v['Type'] == 'nonlearner']

# calculate the vector of the centroid differences.
lcx = [ v[0] - v[1] for v in zip(lax, lbx)]
lcy = [ v[0] - v[1] for v in zip(lay, lby)]
lcz = [ v[0] - v[1] for v in zip(laz, lbz)]

ncx = [ v[0] - v[1] for v in zip(nax, nbx)]
ncy = [ v[0] - v[1] for v in zip(nay, nby)]
ncz = [ v[0] - v[1] for v in zip(naz, nbz)]

# Get size of pairings

cnt_lb = [len(v['UnpairedBefore']) for k,v in sl.items() if v['Type'] == 'learner']
cnt_la = [len(v['UnpairedAfter']) for k,v in sl.items() if v['Type'] == 'learner']
cnt_nb = [len(v['UnpairedBefore']) for k,v in sl.items() if v['Type'] == 'nonlearner']
cnt_na = [len(v['UnpairedAfter']) for k,v in sl.items() if v['Type'] == 'nonlearner']

In [ ]:
#Plot the centroids

learner_bcentroids = go.Scatter3d(
        x=lbx, y=lby, z=lbz,
        mode='markers',
        name='Learner Before centroids',
        text = [ 'Study {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_lb)],
        marker=dict(
            size=5,
            color='rgb(255,0,0)',
            line=dict(
                color='rgb(255,0,0)',
                width=1
            ),
        opacity=0.5
        )
    )

learner_acentroids = go.Scatter3d(
        x=lax, y=lay, z=laz,
        mode='markers',
        name='Learner After centroids',
        text = [ 'Study {0}\\nSize: {1}'.format(v[0], v[1]) for v in zip(learners,cnt_la)],
        marker=dict(
            size=5,
            color='rgb(0,255,0)',
            line=dict(
                color='rgb(0,255,0)',
                width=1
            ),
        opacity=0.5
        )
    )

nonlearner_bcentroids = go.Scatter3d(
        x=nbx, y=nby, z=nbz,
        mode='markers',
        name='Nonlearner Before centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_nb)],
        marker=dict(
            size=5,
            color='rgb(0,0,255)',
            line=dict(
                color='rgb(0,0,255)',
                width=.5
            ),
        opacity=0.5
        )
    )

nonlearner_acentroids = go.Scatter3d(
        x=nax, y=nay, z=naz,
        mode='markers',
        name='Nonlearner After centroids',
        text = [ 'Study: {0}\nSize: {1}'.format(v[0], v[1]) for v in zip(nonlearners,cnt_na)],
        marker=dict(
            size=5,
            color='rgb(128,128,128)',
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

data = [learner_bcentroids, learner_acentroids, nonlearner_acentroids, nonlearner_bcentroids]

layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Locations',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='unpaired-centroid')



In [ ]:
#Plot the centroids

learner_vcentroids = go.Scatter3d(
        x=lcx, y=lcy, z=lcz,
        mode='markers',
        name='Learner Centroid Vectors',
        text=learners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

nonlearner_vcentroids = go.Scatter3d(
        x=ncx, y=ncy, z=ncz,
        mode='markers',
        name='Nonlearner Centroid Vectors',
        text=nonlearners,
        marker=dict(
            size=5,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )


data = [learner_vcentroids, nonlearner_vcentroids]
layout = go.Layout(
            height=1000, width=1000,
            title='Centroid Vectors',
        )

fig = dict(data=data, layout=layout)
    
py.offline.iplot(fig, filename='centroid-vectors')




<H1> General plotting function for synapses

In [ ]:
def plot_synapses(studylist, 
                  tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter'], 
                  slider='Study'):

    data = []
    steps = []
    
    # Generate shades of orange and blue for different traces of the same type (in Trace mode)
    if slider == 'Trace':
        shades = 5
    else:
        shades = 1
    lite_orange = Color("orange")
    lite_orange.saturation = .1
    oranges = list(Color('orange').range_to(lite_orange,shades))

    lite_blue = Color("blue")
    lite_blue.saturation = .1
    blues = list(Color('blue').range_to(lite_orange, shades))
    
    black = list(Color('black').range_to(Color('black'), shades))

    # Set Paired to be shades of orange, Unpaired to be shades of blue
    synapsecolors = {
        'PairedBefore' : oranges,
        'UnpairedBefore' : oranges,
        'AlignedPairedBefore' : oranges,
        'AlignedUnpairedBefore' : oranges,
        'PairedAfter' : blues,
        'UnpairedAfter' : blues,
        'AlignedPairedAfter' : blues,
        'AlignedUnpairedAfter' : blues,
        'AlignmentPts' : black
    }
    
    (outer, inner) = (tracelist, studylist) if slider == 'Trace' else (studylist, tracelist)

    maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
    for o in outer:
        for i in inner:
            (s, t) = (i, o) if slider == 'Trace' else (o, i)
            synapse_data = s[t]

            maxx = max(maxx,synapse_data['x'].max())
            maxy = max(maxy,synapse_data['y'].max())
            maxz = max(maxz,synapse_data['z'].max())
            minx = min(minx,synapse_data['x'].min())
            miny = min(miny,synapse_data['y'].min())
            minz = min(minz,synapse_data['z'].min())
            
            trace_synapses = go.Scatter3d(
                visible = False,
                x=synapse_data['x'], y=synapse_data['y'], z=synapse_data['z'],
                name='{0} {1}'.format(t, s['Study']),
                mode='markers',
                marker=dict(
                    size=4,
                    # Cycle through the shades.  If we are in Study mode, there is only one....
                    color=synapsecolors[t][len(data) % shades].hex,
                    line=dict(
                        color=synapsecolors[t][len(data) % shades].hex,
                        width=0.5
                ),
                opacity=0.4
                )
            )
            data.append(trace_synapses)
            
        
        if slider == 'Trace':
            steps.append(dict(label='{0}'.format(t)))
        else:
            steps.append(dict(label='{0} ({1})'.format(s['Study'],s['Radius'])))
    
    maxx = maxx * 1.02
    maxy = maxy * 1.02
    maxz = maxz * 1.02
    
    mixx = minx * .98
    miny = miny * .98
    mainz = minz * .98

    layout = go.Layout(
                height=900,
                title='Synapses',
                legend=dict(x=.9, xanchor='right'),
                margin=dict(l=20),
                scene=dict(
                    xaxis=dict(title='X Axis', 
                               range=[minx, maxx], 
                               color='red'),
                    yaxis=dict(title='Y Axis', 
                               range=[miny, maxy], 
                               color = 'green'),
                    zaxis=dict(title='Z Axis', 
                               range=[minz, maxz], 
                               color = 'blue',),
                     
                    aspectmode='cube',
                    dragmode='turntable',
                    
                    annotations = [dict(
                        showarrow = False,
                        x = minx, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Medial",
                        font = dict(color = "red",size = 14),
                      ), dict(
                        showarrow = False,
                        x = maxx, y = (maxy-miny)/2+miny, z = (maxz-minz)/2+minz,
                        text = "Lateral",
                        font = dict(color = "red",size = 14),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = miny, z = (maxz-minz)/2+minz,
                        text = "Posterior",
                        opacity = 0.7,
                        font = dict(color = "green",size = 14),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = maxy, z = (maxz-minz)/2+minz,
                        text = "Anterior",
                        xanchor = 'right',
                        font = dict(color = "green",size = 14),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = minz,
                        text = "Dorsal",
                        font = dict(color = "blue",size = 14),
                      ), dict(
                        showarrow = False,
                        x = (maxx-minx)/2+minx, y = (maxy-miny)/2+miny, z = maxz,
                        text = "Ventral",
                        font = dict(color = "blue",size = 14),
                      )]
                )
    )
    
    if slider is False:
        for i in data:
            i['visible'] = True
    else: 
        trace_cnt = len(inner)
        # Set first study to be visible
        for i in range(trace_cnt):
            data[i]['visible'] = True
        
        for i,step in enumerate(steps):
            l = ['visible', [False] * len(data)]
            l[1][i*trace_cnt:i*trace_cnt+trace_cnt] = [True] * trace_cnt
            step['args'] = l
            step['method'] = 'restyle'
        
        sliders = [dict(
            active = 0,
            currentvalue = {"visible": False},
            len = .9,
            steps = steps
        )]

        layout['sliders'] = sliders

    fig = dict(data=data, layout=layout)
    py.offline.iplot(fig, filename='synapse-plot')
    

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study

plot_radius = 4

# Pick which studies we want to plot....


# All studies...
#s_list = [v[plot_study] for k,v in pairs.items() ]

# Just the learners
#s_list = [ v for k,v in syn_pairs[plot_radius].items() if v['Type'] == 'learner']

# Just the learners that have alignment
s_list = [ v for k,v in syn_pairs[plot_radius].items() if v['Study'] in aligned and v['Type'] == 'learner']

# Just the studies in the list of names....
s_list = [ v for k,v in syn_pairs[plot_radius].items() if v['Study'] in ['SynStdR5Y']]
#          ['SynStd6483', 'SynStd7943', 'SynStd7949', 'SynStdR5Y']]
  

# Pick which traces you would like displayed

tracelist =['AlignedPairedBefore','AlignedPairedAfter']
#tracelist = ['PairedBefore','PairedAfter','UnpairedBefore','UnpairedAfter']
#tracelist = ['UnpairedBefore','UnpairedAfter']
print('Plotting {0} studies'.format(len(s_list)))

#Pick the slider mode...

slidermode = 'Study'
#slidermode = 'Trace'
#slidermode = False

#plot_synapses(s_list, tracelist, slidermode, 'auto') 
plot_synapses(s_list, tracelist, slidermode) 